In [36]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import hstack

In [5]:
def save(name, data):
    print "save", name, ":", data
    with open(name, 'w') as f:
        f.write(data)

In [55]:
data_train = pd.read_csv('salary-train.csv')
data_train['FullDescription'] = data_train['FullDescription'].str.lower().replace('[^a-zA-Z0-9]', ' ', regex = True)
data_train['LocationNormalized'].fillna('nan', inplace=True)
data_train['ContractTime'].fillna('nan', inplace=True)

In [74]:
data_test = pd.read_csv('salary-test-mini.csv')
data_test['FullDescription'] = data_test['FullDescription'].str.lower().replace('[^a-zA-Z0-9]', ' ', regex = True)

In [75]:
data_train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,nan,20355


In [93]:
data_test.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,we currently have a vacancy for an hr project ...,Milton Keynes,contract,NaN
1,a web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [66]:
enc = DictVectorizer()
X_train_categ = enc.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [95]:
print X_test_categ

  (0, 0)	1.0
  (0, 1040)	1.0
  (1, 2)	1.0
  (1, 997)	1.0


In [159]:
tfidf = TfidfVectorizer(min_df = 5)
train_vect = tfidf.fit_transform(data_train.FullDescription)
test_vect = tfidf.transform(data_test.FullDescription)

In [160]:
train_features = hstack([train_vect, X_train_categ])
test_features = hstack([test_vect, X_test_categ])

In [163]:
ridge = Ridge(alpha=1)
ridge.fit(train_features, data_train.SalaryNormalized)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [168]:
F = test_features.todense()
res = [ridge.predict(t)[0] for t in F]
print res

[56585.063428573107, 37200.477639017619]


In [176]:
save("1_1", "%.2f %.2f" % tuple(res))

save 1_1 : 56585.06 37200.48
